In [8]:
import pandas as pd
df_tennis = pd.DataFrame(data=pd.read_csv('Datasets.csv'))
print(df_tennis)

def entropy(probs):
    import math
    return sum([-prob*math.log(prob,2) for prob in probs]) 

def entropy_of_list(a_list):
    from collections import Counter
    cnt = Counter(x for x in a_list)
    print("No and Yes class:",a_list.name,cnt)
    num_instances = len(a_list)*1.0
    probs = [x/num_instances for x in cnt.values()]
    return entropy(probs)
print(df_tennis['playtennis'])
total_entropy = entropy_of_list(df_tennis['playtennis'])
print("Entropy of given playtennis dataset:",total_entropy)

def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    print("Info gain calculation of ",split_attribute_name)
    df_split = df.groupby(split_attribute_name)
    for name, group in df_split:
        print(name)
        print(group)
    nobs = len(df.index)*1.0
    df_agg1 = df_split.agg({target_attribute_name:lambda x:entropy_of_list(x)})
    df_agg2 = df_split.agg({target_attribute_name:lambda x:len(x)/nobs})
    df_agg1.columns=['entropy']    
    df_agg2.columns=['proportion']
    new_entropy = sum(df_agg1['entropy']*df_agg2['proportion'])
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy-new_entropy

print("Info gain for outlook is: "+str(information_gain(df_tennis,'outlook','playtennis')),"\n")    
print("Info gain for humidity is: "+str(information_gain(df_tennis,'humidity','playtennis')),"\n")    
print("Info gain for temperature is: "+str(information_gain(df_tennis,'temperature','playtennis')),"\n")    

def id3(df,target_attribute_name, attribute_name,default_class=None):
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])
    if len(cnt)==1:
        return next(iter(cnt))
    elif df.empty or (not attribute_name):
        return default_class
    else:
        default_class = max(cnt.keys())
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_name]
        index_of_max = gainz.index(max(gainz))
        best_attr = attribute_name[index_of_max]
        tree = {best_attr:{ }}
        remainig_attribute_names = [i for i in attribute_name if i!=best_attr]
    for attr_val, data_subset in df.groupby(best_attr):
        subtree = id3(data_subset, target_attribute_name,remainig_attribute_names,default_class)
        tree[best_attr][attr_val] = subtree
    return tree

attribute_names = list(df_tennis.columns)
print("List of attributes:", attribute_names)
attribute_names.remove('playtennis')
print("predicting attributes:", attribute_names)

from pprint import pprint
tree = id3(df_tennis,"playtennis", attribute_names)
pprint("\n\n The result decision tree is: \n")
pprint(tree)

    outlook temperature humidity    wind playtennis
0     sunny         hot     high    weak         no
1     sunny         hot     high  strong         no
2  overcast         hot     high    weak        yes
3      rain        mild     high    weak        yes
4      rain        cool   normal    weak        yes
5      rain        cool   normal  strong         no
6  overcast        cool   normal  strong        yes
7     sunny        mild     high    weak         no
8     sunny        cool   normal    weak        yes
9      rain        mild   normal    weak        yes
0     no
1     no
2    yes
3    yes
4    yes
5     no
6    yes
7     no
8    yes
9    yes
Name: playtennis, dtype: object
No and Yes class: playtennis Counter({'yes': 6, 'no': 4})
Entropy of given playtennis dataset: 0.9709505944546686
Info gain calculation of  outlook
overcast
    outlook temperature humidity    wind playtennis
2  overcast         hot     high    weak        yes
6  overcast        cool   normal  strong     